# MBTI함수(거래년월, 고객ID가 input으로 들어오면 최종 MBTI 도출)

In [1]:
import pandas as pd
import numpy as np

def create_E_I_dataframe(df, w, x, y):
    # 거래년월 목록 추출
    unique_months = df['거래년월'].unique()
    
    # 결과를 저장할 빈 리스트 생성
    result_list = []
    
    for month in unique_months:
        # 해당 거래년월의 데이터 필터링
        df_month = df[df['거래년월'] == month]
        
        # 고객ID별 가맹점업종명 고유 개수와 승인건수 합계 계산
        df_group = df_month.groupby('고객ID')['가맹점업종명'].nunique().reset_index()
        a = df_month.groupby('고객ID')['승인건수_수치형'].sum().reset_index()
        df_group['승인건수_총합'] = a['승인건수_수치형']
        df_group['다양성지수'] = df_group['가맹점업종명'] / df_group['승인건수_총합']
        
        # 다양성 분류 초기화
        df_group['다양성분류'] = np.where(df_group['다양성지수'] > w, 'E', 'I')
        
        # 조건에 따른 분류 업데이트
        cond1 = df_group['가맹점업종명'] < x
        df_group.loc[cond1, '다양성분류'] = 'I'
        cond2 = df_group['가맹점업종명'] > y
        df_group.loc[cond2, '다양성분류'] = 'E'
        
        # 결과에 거래년월 추가
        df_group['거래년월'] = month
        
        # 필요한 컬럼만 선택하여 리스트에 추가
        result_list.append(df_group[['거래년월', '고객ID', '다양성분류']])
    
    # 결과 리스트를 하나의 DataFrame으로 병합
    result_df = pd.concat(result_list, ignore_index=True)
    result_df.rename(columns={'다양성분류': 'E_I'}, inplace=True)
    
    return result_df


In [2]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'AppleGothic'

pd.options.display.float_format = '{:.2f}'.format
# 특정 모듈의 경고 메시지 무시
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
df = pd.read_csv('final_card.csv')

In [4]:
# 승인건수를 수치형 값으로 변환하는 매핑 사전 생성
mapping = {
    "5번이하": 1,
    "5번초과 10번이하": 6,
    "10번초과 15번이하": 11,
    "15번초과 20번이하": 16,
    "20번 초과": 21
}

# 승인건수 컬럼의 값을 매핑
df["승인건수_수치형"] = df["승인건수"].map(mapping)

# 결과 확인
df

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액,승인건수_수치형
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000,1
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000,1
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000,1
3,202112,2950007638,일반한식,경상북도,경주시,5번이하,60000,1
4,202112,2931907302,일반한식,경상북도,경주시,5번이하,10000,1
...,...,...,...,...,...,...,...,...
188447864,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000,1
188447865,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000,1
188447866,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000,1
188447867,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000,1


In [5]:
pivot = df.pivot_table(index=['거래년월', '고객ID'], 
                       values='승인건수_수치형', 
                       aggfunc='sum').reset_index()

# 2. 승인건수_수치형 값이 2보다 작은 경우 필터링
to_drop = pivot[pivot['승인건수_수치형'] <= 5][['거래년월', '고객ID']]

# 3. 원래 데이터프레임에서 삭제
df_filtered = df.merge(to_drop, on=['거래년월', '고객ID'], how='left', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
df_filtered

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액,승인건수_수치형
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000,1
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000,1
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000,1
3,202112,2950007638,일반한식,경상북도,경주시,5번이하,60000,1
4,202112,2931907302,일반한식,경상북도,경주시,5번이하,10000,1
...,...,...,...,...,...,...,...,...
188447864,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000,1
188447865,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000,1
188447866,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000,1
188447867,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000,1


In [6]:
# 모든 카테고리를 포함한 분류 매핑
category_mapping = {
    '실용': {
        '음식_실용': [
            '정육점', '스넥', '제과점', '기타음료식품', '농·축·수산품', '홍삼제품', 
            '기타건강식품', '인삼제품','위탁급식업'
        ],
        '교통_실용': [
            '택시', '철도', '고속버스', '택시회사', 'SK주유소', 'GS주유소', '쌍용S-OIL', 
            '현대정유(오일뱅크)', 'LPG취급점', '전기차충전소', '유류판매', 'E1가스충전소', 
            'SK가스충전소', '현대정유가스충전소', '주유소', '자동차부품', '가타자동차서비스', 
            '세차장', '자동차정비', '자동차시트·타이어', '윤활유전문판매', '화물운송', 
            '이륜차판매', 'GS가스충전소', '쌍용S-OIL가스충전소'
        ],
        '의료': [
            '종합병원', '치과의원', '치과병원', '한방병원', '동물병원', '의원', '의 원', 
            '건강진단', '약 국', '한약방', '의료용품', '기타의료기관 및 기타의료기기','한 의 원', '병 원(응급실운영)'
        ],
        '쇼핑_실용': [
            '슈퍼마켓', '편의점', '농협하나로클럽', '농,축협직영매장', '화원', 
            '인터넷Mall', 'CATV홈쇼핑', '통신판매업1', '인터넷P/G', '인터넷종합Mall','일반(통신판매)',
            '가전제품', '컴퓨터', '통신기기', '사무용 OA기기', '문구용품', 'PG상품권',
            '주방용구', '주방용식기', '편 의 점','안경','기타유통업', '일반가구', '자사카드발행백화점',
            '미용재료','기타 사무용품','상품권전문판매', '상 품 권', '전자상거래상품권','전자상거래상품권전문판매'
        ],
        '부동산/건축': [
            '부동산중개·임대', '페인트', '조명기구', '목재·석재·철물', '건축용 요업제품', 
            '냉열기기', '보일러·펌프·샷시', '기타건축자재'
        ],
        '교육': [
            '보습학원', '예·체능계학원', '기능학원', '외국어학원', '초중고교육기관', 
            '유아원', '독서실', '학습지교육', '기타 교육기관', '학원'
        ],
        '서비스_실용': [
            '세탁소', '가정용품수리', '주차장', '기타수리서비스', '기타용역서비스',
            '보관및 창고업', '혼례서비스업', '정수기', '가례서비스업', '사무서비스', '종합용역',
            '미용원', '이용원', '기타대인서비스', '조세서비스', '위성방송', 'CATV'
        ],
        '농업/농자재': [
            '농기계', '비료,사료,종자', '농축수산가공품', '미곡상', '기타농업관련'
        ],
        '금융/보험': ['손해보험', '생명보험', '기타보험', '비씨카드 정산용가맹점', '비씨카드 정산용(할인)'],
        '공공서비스': [
            '공공요금대행서비스/소득공제대상', '공공요금대행서비스/소득공제비대상', 
            '구내매점(국가기관등)', '비영리/대상', '비영리/비대상', '통신서비스/소득공제비대상'
        ],
        '통신/IT': ['소프트웨어', '사무·통신기기수리', '이동통신요금'],
        '기계/공구': ['기계공구', '카인테리어', '기타전기제품', 'DP&E']
        #'반려동물': ['애완동물', '수족관']
    },
    
    '감성': {
        '음식_감성': [
            '일반한식', '갈비전문점', '한정식', '중국식', '일식·회집', '서양음식', '주류판매점'
        ],
        '레저/스포츠': [
            '골프경기장', '스크린골프', '볼링장', '수영장', '당구장', '골프연습장', 
            '사우나', '헬스클럽', '안마/스포츠마사지', '스포츠·레져용품', 
            '종합레져타운', '레져업소', '관광여행', '기타레져업소', '노 래 방', '볼 링 장', '수 영 장',
            '골프용품 전문점'
        ],
        '쇼핑_감성': [
            '기념품점', '기타잡화', '정장', '스포츠의류', '아동의류', '신발', '맞춤복점', 
            '화장품','귀금속', '대형할인점', '면 세 점', '침구·수예점',
            '기타의류', '가 방', '신 발', '옷감·직물', '카페트,커튼,천막,지물', '기타가구', '액세서리', 
            '내의판매', '단체복', '연 쇄 점', '캐주얼의류', '제 화'
        ],
        '교통_감성': [
            '여객선', '렌터카', '항공사'
        ],
        '문화': [
            '서점', '영화관', '티켓', '민예·공예품', '화방·표구점', '전문서적', 
            '일반서적', '문화취미기타', '출판 및 인쇄물', '기타서적문구', '완구점', '시 계', '음반영상물'
        ],
        '유흥': [
            '주점', '유흥주점', '단란주점', '칵테일바', '스파'
        ],
        '숙박': [
            '특급호텔', '1급 호텔', '2급 호텔', '여관', '모텔', '콘도', 
            '기타숙박업'
        ],
        '서비스_감성': [
            '피부미용실', '인테리어전문'
        ],
        '반려동물': ['애완동물', '수족관']
    },
    '기타': {
        '기타': [
            '업종미등록', '기타잡화', '기타1', '기타4', '기타 전문점'
        ]
    }
}


In [7]:
upjong_list = [
    '일반한식', '자동차정비', '미용원', '기타레져업소', '기념품점', '약 국', '애완동물', '슈퍼마켓',
       '주점', '유흥주점', '서양음식', 'SK주유소', '정육점', '스넥', '의 원', '농·축·수산품',
       '중국식', '한 의 원', '스포츠·레져용품', '보습학원', '기타음료식품', '쌍용S-OIL', '제과점',
       '통신기기', '화원', '화장품', '기타잡화', '유아원', '기타숙박업', 'GS주유소', '편 의 점',
       'LPG취급점', '철도', '페인트', '출판 및 인쇄물', '정장', '농협하나로클럽', '농,축협직영매장',
       '귀금속', '문구용품', '일식·회집', '주유소', '피부미용실', '티켓', '위탁급식업', '예·체능계학원',
       'SK가스충전소', '비료,사료,종자', '침구·수예점', '자동차부품', '당구장', '가전제품', '인터넷Mall',
       '스포츠의류', '현대정유(오일뱅크)', '가정용품수리', '사무·통신기기수리', '안경', '세탁소', '사우나',
       '일반서적', '노 래 방', '스크린골프', '기계공구', '인터넷P/G', '완구점', '기타의류', '미곡상',
       '기타건강식품', '기타용역서비스', '치과의원', '헬스클럽', '기타수리서비스', '기타건축자재', '일반가구',
       '골프경기장', '병 원(응급실운영)', '자동차시트·타이어', '주방용구', '연 쇄 점', '액세서리',
       '전문서적', '주차장', '특급호텔', '1급 호텔', '치과병원', '민예·공예품', '건축용 요업제품',
       '단란주점', '옷감·직물', '자사카드발행백화점', '인삼제품', '외국어학원', '동물병원', '대형할인점',
       '독서실', '농기계', '조명기구', '세차장', '사무서비스', '컴퓨터', '렌터카', '가 방', 'PG상품권',
       '기타 교육기관', '갈비전문점', '보일러·펌프·샷시', '기능학원', '카페트,커튼,천막,지물', '기타유통업',
       '이륜차판매', '종합레져타운', '카인테리어', '화물운송', '내의판매', '종합병원', '항공사',
       '골프용품 전문점', '목재·석재·철물', '홍삼제품', '의료용품', '기타 사무용품', '미용재료', '기타가구',
       '골프연습장', '신 발', '손해보험', '영화관', '비영리/비대상', '기타대인서비스', '기타전기제품',
       'DP&E', '주방용식기', '가타자동차서비스', '비씨카드 정산용가맹점', '기타농업관련',
       '구내매점(국가기관등)', '아동의류', '이용원', '상 품 권', '조세서비스', '사무용 OA기기',
       '문화취미기타', '인테리어전문', 'GS가스충전소', '쌍용S-OIL가스충전소', '가례서비스업', '시 계',
       '기타의료기관 및 기타의료기기', '비영리/대상', '현대정유가스충전소', '윤활유전문판매', '정수기',
       '공공요금대행서비스/소득공제비대상', '화방·표구점', '볼 링 장', '한약방', '인터넷종합Mall',
       '기타 전문점', '콘도', '통신판매업1', '종합용역', '주류판매점', '2급 호텔', '학습지교육',
       '전기차충전소', '건강진단', '유류판매', '한방병원', '업종미등록', '이동통신요금',
       '통신서비스/소득공제비대상', '기타서적문구', '수족관', 'E1가스충전소', '보관및 창고업', '여객선',
       '택시', 'CATV홈쇼핑', '수 영 장', '농축수산가공품', '소프트웨어', '생명보험', '면 세 점',
       '칵테일바', '관광여행', '기타보험', '초중고교육기관', '비씨카드 정산용(할인)', '고속버스',
       '일반(통신판매)', '레져업소', '기타4', '학원', '기타1', '공공요금대행서비스/소득공제대상',
       '안마/스포츠마사지', '부동산중개·임대', '한정식', '상품권전문판매', '혼례서비스업', '냉열기기',
       '맞춤복점', '단체복', '전자상거래상품권','택시회사', '전자상거래상품권전문판매', '위성방송', 'CATV', '제 화', '캐주얼의류', '음반영상물'
]

In [8]:
import pandas as pd

# 매핑 함수
def map_category(upjong):
    for main_category, subcategories in category_mapping.items():
        for sub_category, keywords in subcategories.items():
            if upjong in keywords:
                return main_category, sub_category
    raise ValueError(f"매핑되지 않은 항목: {upjong}")

# 매핑 실행
mapped_data = []
for upjong in upjong_list:
    try:
        main_category, sub_category = map_category(upjong)
        mapped_data.append([upjong, main_category, sub_category])
    except ValueError as e:
        print(e)

# 결과 저장
df1 = pd.DataFrame(mapped_data, columns=['가맹점업종명', '실용/감성 분류', '세분류'])

# 저장
# df.to_csv('mapped_upjong.csv', index=False, encoding='utf-8-sig')
df1.head()

,가맹점업종명,실용/감성 분류,세분류
0,일반한식,감성,음식_감성
1,자동차정비,실용,교통_실용
2,미용원,실용,서비스_실용
3,기타레져업소,감성,레저/스포츠
4,기념품점,감성,쇼핑_감성


In [9]:
import pandas as pd
import numpy as np

def calculate_mbti(df_filtered, df1, 거래년월, 고객ID):
    w = 0.5
    x = 5
    y = 25

    # 거래년월에 해당하는 데이터 확인
    if 거래년월 not in df_filtered['거래년월'].unique():
        return f"Error: 거래년월 {거래년월}에 해당하는 데이터가 없습니다."

    # 거래년월 및 고객ID로 필터링
    df_month = df_filtered[(df_filtered['거래년월'] == 거래년월) & (df_filtered['고객ID'] == 고객ID)]

    # 고객ID 확인
    if df_month.empty:
        return f"Error: 거래년월 {거래년월}에 고객ID {고객ID}에 해당하는 데이터가 없습니다."

    # E, I 계산
    df_group = df_month.groupby('고객ID')['가맹점업종명'].nunique().reset_index()
    a = df_month.groupby('고객ID')['승인건수_수치형'].sum().reset_index()
    df_group['승인건수_총합'] = a['승인건수_수치형']
    df_group['다양성지수'] = df_group['가맹점업종명'] / df_group['승인건수_총합']

    # 다양성 분류
    df_group['다양성분류'] = np.where(df_group['다양성지수'] > w, 'E', 'I')
    cond1 = df_group['가맹점업종명'] < x
    df_group.loc[cond1, '다양성분류'] = 'I'
    cond2 = df_group['가맹점업종명'] > y
    df_group.loc[cond2, '다양성분류'] = 'E'

    df_group['거래년월'] = 거래년월
    df_group['고객ID'] = 고객ID

    result_df = df_group[['거래년월', '고객ID', '다양성분류']]
    result_df.rename(columns={'다양성분류': 'E_I'}, inplace=True)

    # N, S 계산
    diversity = df_month.groupby(['거래년월', '고객ID'])['가맹점_광역시도'].nunique().reset_index()
    diversity.rename(columns={'가맹점_광역시도': '가맹점_광역시도_다양성'}, inplace=True)
    diversity['S_N'] = diversity['가맹점_광역시도_다양성'].apply(lambda x: 'S' if x >= 3 else 'N')
    diversity = diversity[['거래년월', '고객ID', 'S_N']]

    # F, T 계산
    df2 = df1[['가맹점업종명', '실용/감성 분류']]
    df_result = pd.merge(df_month, df2, on='가맹점업종명', how='left')

    df_실용_감성_sum = df_result.pivot_table(
        index=['거래년월', '고객ID'],
        columns='실용/감성 분류',
        values='승인금액',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    df_실용_감성_sum.columns.name = None
    df_filtered_실용_감성_승인금액 = df_result.pivot_table(
        index=['거래년월', '고객ID'],
        values='승인금액',
        aggfunc='sum'
    ).reset_index()

    df_실용_감성_final = pd.merge(df_실용_감성_sum, df_filtered_실용_감성_승인금액, on=['거래년월', '고객ID'])
    df_실용_감성_final['실용_비율'] = df_실용_감성_final['실용'] / df_실용_감성_final['승인금액']
    df_실용_감성_final['T_F'] = df_실용_감성_final['실용_비율'].apply(lambda x: 'T' if x > 0.72 else 'F')
    df_실용_감성_final = df_실용_감성_final[['거래년월', '고객ID', 'T_F']]

    # P, J 계산
    df_grouped = df_filtered[df_filtered['고객ID'] == 고객ID].groupby(['고객ID', '거래년월'], as_index=False)['승인금액'].sum()
    df_grouped = df_grouped.sort_values(by=['고객ID', '거래년월'])
    df_grouped['직전달_승인금액'] = df_grouped.groupby('고객ID')['승인금액'].shift(1)

    # 삭제된 행 확인
    missing_rows = df_grouped[df_grouped['직전달_승인금액'].isna() & (df_grouped['거래년월'] == 거래년월)]
    if not missing_rows.empty:
        return f"Error: 고객ID {고객ID}의 거래년월 {거래년월}에 대해 직전달 정보가 없어 변동값을 계산할 수 없습니다."

    df_grouped['변동성(%)'] = ((df_grouped['승인금액'] - df_grouped['직전달_승인금액']) / df_grouped['직전달_승인금액']) * 100
    df_grouped = df_grouped.dropna()
    b = df_grouped[df_grouped['거래년월'] == 거래년월]
    b['J_P'] = b['변동성(%)'].apply(lambda x: 'P' if x < -31.944444444444443 or x > 48.598130841121495 else 'J')
    b = b[['고객ID', '거래년월', 'J_P']]

    # 최종 MBTI 도출
    result_1 = pd.merge(result_df, diversity, on=['거래년월', '고객ID'])
    result_2 = pd.merge(result_1, df_실용_감성_final, on=['거래년월', '고객ID'])
    result_3 = pd.merge(result_2, b, on=['거래년월', '고객ID'])
    result_3['MBTI'] = result_3['E_I'] + result_3['S_N'] + result_3['T_F'] + result_3['J_P']

    return result_3


In [10]:
result = calculate_mbti(df_filtered, df1, 202112, 3857466791)
result

,거래년월,고객ID,E_I,S_N,T_F,J_P,MBTI
0,202112,3857466791,I,N,F,J,INFJ
